In [2]:
import pickle
import cv2
import matplotlib.pyplot as plt
from CollageBuilder import build_collage, get_ocr, get_piece_number
from ExtractionModel import *
from tqdm.notebook import tqdm
%matplotlib qt

Segmentation Models: using `keras` framework.


In [ ]:
def undistort(img):

    roi = (125, 102, 860, 515)

    mtx = np.array([[607.43997952,   0.        , 645.96889836],
                    [  0.        , 605.58954233, 391.50563407],
                    [  0.        ,   0.        ,   1.        ]])

    dist = np.array([[-0.35962477,  0.16171162, -0.00081826, -0.00444003, -0.03848117]])

    newcameramtx = np.array([[408.22506714,   0.        , 558.75454709],
                          [  0.        , 390.17321777, 354.4384903 ],
                          [  0.        ,   0.        ,   1.        ]])

    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
    x, y, w, h = roi
    d = dst.copy()
    dst = dst[y:y+h, x:x+w]
    return dst


In [ ]:
with open('label', "rb") as fp:
    frames = pickle.load(fp)    
ims = [cv2.cvtColor(cv2.imdecode(im[1], cv2.IMREAD_UNCHANGED), cv2.COLOR_BGR2RGB) for im in frames]
# shape = ims[0].shape[:2][::-1]
# undistorted_ims = [cv2.resize(undistort(im), shape) for im in tqdm(ims)]

In [ ]:
# for i in range(len(ims)):
#     cv2.imshow('distorted', np.hstack((ims[i], undistorted_ims[i])))
#     cv2.waitKey(50)

In [10]:
mov = cv2.VideoCapture('IMG_4280.MOV')
if (mov.isOpened() == False):
    print("Error opening the video file")
else:
    fps = int(mov.get(5))
    print("Frame Rate : ",fps,"frames per second") 
    frame_count = mov.get(7)
    print("Frame count : ", frame_count)

In [4]:
from IPython.display import clear_output
frames = []
for i in tqdm(range(int(frame_count))):
    ret, frame = mov.read()
    if ret == True:
        frames.append(frame)
    else:
        continue
    break

  0%|          | 0/5938 [00:00<?, ?it/s]

array([[[ 63,  70,  80],
        [ 68,  75,  84],
        [ 74,  78,  90],
        ...,
        [106, 112, 124],
        [105, 111, 123],
        [105, 111, 123]],

       [[ 65,  72,  81],
        [ 69,  76,  86],
        [ 74,  78,  90],
        ...,
        [108, 114, 126],
        [108, 114, 126],
        [109, 115, 127]],

       [[ 68,  72,  82],
        [ 73,  76,  87],
        [ 76,  79,  91],
        ...,
        [112, 118, 130],
        [114, 120, 132],
        [116, 122, 134]],

       ...,

       [[ 46,  37,  38],
        [ 45,  35,  37],
        [ 51,  39,  42],
        ...,
        [ 95,  95, 100],
        [ 94,  94,  99],
        [ 86,  86,  91]],

       [[ 41,  30,  31],
        [ 40,  28,  30],
        [ 45,  33,  36],
        ...,
        [ 91,  91,  96],
        [ 89,  88,  94],
        [ 85,  84,  91]],

       [[ 34,  23,  24],
        [ 34,  23,  24],
        [ 40,  28,  31],
        ...,
        [ 85,  85,  90],
        [ 80,  79,  85],
        [ 78,  77,  84]]

In [ ]:
ext = ExtractionModel()
ext.set_mode(ExtractMode.LABEL)

In [ ]:
%%time
masks, labels = zip(*[ext.extract(im) for im in tqdm(ims)])
masks = list(cv2.cvtColor(m, cv2.COLOR_GRAY2BGR) for m in masks)
# labels = list(labels)
labels = list([cv2.rotate(label, cv2.ROTATE_90_CLOCKWISE) for label in labels])
# collage = build_collage(labels)

# ocr = get_ocr(collage)
# top_val, bottom_val = get_piece_number(ocr)

# print(f'Piece Identification: {top_val} {bottom_val}')

In [ ]:
# labels = list([cv2.rotate(label, cv2.ROTATE_90_COUNTERCLOCKWISE) for label in labels])

In [ ]:
collages = [build_collage(labels[:i]).astype(np.uint8) for i in range(len(labels))]

In [ ]:
results = [get_piece_number(col) for col in tqdm(collages)]

In [ ]:
h = 500
w = int(h*1.7777777777)

In [ ]:
top = sorted(results[-1][0].items(), key=lambda x: -x[1])[0]
# top = ('', '')
bottom = sorted(results[-1][1].items(), key=lambda x: -x[1])[0]
# bottom = ('', '')

In [ ]:
def get_res_img(h, w, res):
    im = np.zeros((h, w, 3)).astype(np.uint8)
    i = 100
    cv2.putText(im, f'Top Row', (50, 50), 2, .75, (255, 255, 255), 1)
    cv2.putText(im, f'Bottom Row', (500, 50), 2, .75, (255, 255, 255), 1)
    for value, count in sorted(res[0].items(), key=lambda x: -x[1]):
        cv2.putText(im, f'{value}', (50, i), 2, .75, (255, 255, 255), 1)
        cv2.putText(im, f'{count}', (125, i), 2, .75, (255, 255, 255), 1)
        i += 20
    i = 100
    for value, count in sorted(res[1].items(), key=lambda x: -x[1]):
        cv2.putText(im, f'{value}', (500, i), 2, .75, (255, 255, 255), 1)
        cv2.putText(im, f'{count}', (575, i), 2, .75, (255, 255, 255), 1)
        i += 20
    return im

In [ ]:
demo = []
for i in range(len(labels)):
    left = np.vstack((cv2.cvtColor(cv2.resize(ims[i], (w, h)), cv2.COLOR_BGR2RGB), cv2.resize(masks[i], (w, h))))
    right = np.vstack((cv2.resize(cv2.cvtColor(build_collage(labels[:i+1]).astype(np.uint8), cv2.COLOR_BGR2RGB), (w, h)),
                     get_res_img(h, w, results[i]))).astype(np.uint8)
    final = np.hstack((left, right))
    demo.append(final)
final_results_img = get_res_img(h, w, results[-1])
cv2.putText(final_results_img, f'Piece Identification: {top[0]} {bottom[0]}', (50, h-25), 2, 1.25, (0, 255, 0), 1)
left = np.vstack((cv2.cvtColor(cv2.resize(ims[-1], (w, h)), cv2.COLOR_BGR2RGB), cv2.resize(masks[-1], (w, h))))
right = np.vstack((cv2.resize(cv2.cvtColor(build_collage(labels).astype(np.uint8), cv2.COLOR_BGR2RGB), (w, h)),
                 final_results_img)).astype(np.uint8)
final = np.hstack((left, right))
demo.append(final)

In [ ]:
for frame in demo:
    cv2.imshow('demo', frame)
    cv2.waitKey(50)